In [ ]:
import numpy as np
import pandas as pd
import googlemaps
gmaps = googlemaps.Client(key = 'Your_API_key')
import time
from datetime import datetime
import math
from math import sin, cos, sqrt, atan2, radians

In [ ]:
# Define a function for calculating the great-circle distance based on geocoordinates
def H_dist(origin,dest):
    R = 6378.137
    flattening = 1/298.257223563
    lat1 = atan2((1-flattening)*sin(origin[0]*math.pi/180), cos(origin[0]*math.pi/180))
    lon1 = origin[1]*math.pi/180
    lat2 = atan2((1-flattening)*sin(dest[0]*math.pi/180), cos(dest[0]*math.pi/180))
    lon2 = dest[1]*math.pi/180
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [ ]:
# Import geocoordinate information for world’s busiest airports by passenger traffic
hub_location = pd.read_csv('Alternative hub locations.csv')
# Import geocoordinate information for participants
df = pd.read_csv('Geocoordinate.csv')
# Data processing for alternative hub locations for 1-hub scenarios
location_1hub = hub_location[hub_location.Rank<=30]
location_1hub = location_1hub.reset_index(drop=True)
# Data processing for alternative hub locations for multi-hub scenarios
location_multihub = hub_location[(hub_location.Rank<=30) | ((hub_location.Rank<=50) & (hub_location.Country=='United States '))]
location_multihub = location_multihub.reset_index(drop=True)
# Create dataframe for transportation mode and distances computed in the following
rowname = location_multihub.index.values
mode = pd.DataFrame(columns = rowname,index = df.index.values) 
dist = pd.DataFrame(columns = rowname,index = df.index.values) 

In [ ]:
# Define time for the depature time
dt = datetime.strptime('8 Mar 2021', '%d %b %Y')
newdatetime = dt.replace(hour=9, minute=0)

# Computation of distances between alternative hub locations and the participant's locations
# Thresholds of 600-km by rail and 500-km by car are not applied
# Variable "location_multihub" can be replaced by "location_1hub" to calculate distances for 1-hub scenarios
for i in range(len(location_multihub)):
    origin = (location_multihub.lat[i],location_multihub.long[i])
    for j in range(len(df)):
        dest = (df.lat[j],df.long[j])
        try:
            # For european countries, calculate both rail and car transportation distances
            if(df.iloc[j,4] in ['BE','CH','DE','ES','RER / FR','GB','IT','NL','SE']):
                # distance for rail transport in Europe
                matrix1 = gmaps.distance_matrix(origin, dest, mode = 'transit', transit_mode = 'rail', departure_time = newdatetime)
                matrixdf1 = pd.json_normalize(matrix1, ['rows','elements'])
                # distance for car transportation in Europe
                matrix2 = gmaps.distance_matrix(origin, dest, mode = 'driving')
                matrixdf2 = pd.json_normalize(matrix2, ['rows','elements'])
                # Exception: if rail transport distance > 600 km and car transportation distance < 500 km, then choose driving instead of rail, else if rail transport distance < 600 km, choose rail.
                if(matrixdf1['distance.value'].values[0]>600000 and matrixdf2['distance.value'].values[0]<=500000):
                    print(matrixdf1['distance.value'].values[0],'use driving instead of rail',matrixdf2['distance.value'].values[0])
                    mode.iloc[j,i] = 'drive'
                    dist.iloc[j,i] = matrixdf2['distance.value'].values[0]/1000
                else:
                    print(matrixdf1['distance.value'].values[0])
                    mode.iloc[j,i] = 'rail'
                    dist.iloc[j,i] = matrixdf1['distance.value'].values[0]/1000
            # For regions outside the Europe, calculate car transportation distances only           
            else:
                matrix = gmaps.distance_matrix(origin, dest, mode = 'driving')
                matrixdf = pd.json_normalize(matrix, ['rows','elements'])
                print(matrixdf['distance.value'].values[0])
                mode.iloc[j,i] = 'drive'
                dist.iloc[j,i] = matrixdf['distance.value'].values[0]/1000
        except IndexError:
            print("Index was wrong...")
        except ValueError:
            print("Unexpected value...")
        except:
            print("invalid request")
            print(H_dist(origin,dest))
            dist.iloc[j,i] = H_dist(origin,dest)
            mode.iloc[j,i] = 'No route'

In [ ]:
Participant_nearest_airport = pd.read_csv('Participant_nearest_airport.csv')
Participant_nearest_airport.head()

dist_new = dist.copy()
mode_new = mode.copy()

# Computation of distances between alternative hub locations and the participant's locations
# Thresholds of 600-km by rail and 500-km by car are applied
# Variable "location_multihub" can be replaced by "location_1hub" to calculate distances for 1-hub scenarios
for i in range(len(location_multihub)):
    origin = (location_multihub.lat[i],location_multihub.long[i])
    for j in range(len(df)):
        dest = (Participant_nearest_airport.Dest_lat[j],Participant_nearest_airport.Dest_long[j])
        # 500-km threshold for driving: if flying is required, then total distance = great-circle distance between the origin airport and destination airport + ground transportation distance between participant's location and the origin airport
        if ((dist_new.iloc[j,i] > 500) & (mode_new.iloc[j,i]=='drive')):
            dist_new.iloc[j,i] = H_dist(origin, dest)+Participant_nearest_airport.iloc[j,9]
        # 600-km threshold for rail: if flying is required, then total distance = great-circle distance between the origin airport and destination airport + ground transportation distance between participant's location and the origin airport
        if ((dist_new.iloc[j,i] > 600) & (mode_new.iloc[j,i]=='rail')):
            dist_new.iloc[j,i] = H_dist(origin, dest)+Participant_nearest_airport.iloc[j,9]
        # if no ground transportation route between alternative hub locations and the participant's locations, then calculate as above
        if(mode_new.iloc[j,i]=='No route'):
            dist_new.iloc[j,i] = H_dist(origin, dest)+Participant_nearest_airport.iloc[j,9]
        # if distance between alternative hub locations and the participant's locations satisfies the thresholds, then keep the original value
        else:
            dist_new.iloc[j,i] = dist_new.iloc[j,i]

# Export distance matrix for the optimization model
#dist_new.iloc[:,0:30].to_csv(r'Distance_matrix_1hub.csv')
dist_new.to_csv(r'Distance_matrix_multihub.csv')